In [3]:
# important modules for this HW
import bs4 # doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests as rq 
import re
import time
# previous useful modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [4]:
import copy
import datetime

# Loading the 2 dataset

In [ ]:
df1=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_1.csv')

In [ ]:
df2=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_2.csv')

In [ ]:
df=pd.concat([df1,df2],axis=0)

In [ ]:
df.iloc[0].link

In [ ]:
df.head()

In [ ]:
df[df.nom=='Baumer Peter']

In [ ]:
a=list(df.cat.value_counts().index)

In [ ]:
a

In [ ]:
st=""
for i,item in enumerate(a):
    if i>1000 and i<1500:
        st+=item
        st+=" "
st

In [ ]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

# Links2run integration

### Loading links2run

In [ ]:
links2runs=pd.read_csv('../datasets/links2runs.csv')
del links2runs['Unnamed: 0']

In [ ]:
links2runs=links2runs.set_index(links2runs.URL)

In [ ]:
links2runs.head(3)

In [ ]:
links2runs.loc['http://services.datasport.com/2000/lauf/tdcn']

In [ ]:
links2runs.index.value_counts()

### Process link

In [ ]:
df.link=df.link.apply(lambda x: '/'.join(x.split('/')[:-1]))

In [ ]:
df.head(2)

### Groupby link

In [ ]:
links=list(df.link.value_counts().index)

In [ ]:
def add_date_name_to_group(x):
    link=x.link.iloc[0]
#     print(link)
#     print('\n\n\n\n')
    row_links2runs=links2runs.loc[link]
    try:
        row_links2runs.shape[1]
        if row_links2runs.shape[0]>1:
            row_links2runs=row_links2runs.iloc[0]
    except:
        pass
#     print(row_links2runs.shape)
    try:
        x['Date']=row_links2runs.Date
    except:
        print(row_links2runs)
        print(x)
        raise()
    x['Name']=row_links2runs.Name
    return x

In [ ]:
df.head(2)

In [ ]:
grouped_link=df.groupby(df.link)

In [ ]:
df=grouped_link.apply(add_date_name_to_group)

In [ ]:
del df['link']

In [ ]:
df.head(2)

# Delete duplicates
Some links of games are counted twice. In this case we check for duplicated rows and we remove them

In [ ]:
df=df.drop_duplicates(keep='first')

# Process temps-retards

In [ ]:
class Time:
    def __init__(self,time,mseconds=True):
        self.days,self.hours,self.minutes,self.seconds,self.mseconds=self.parse_time(time,split=True,mseconds=mseconds)
        
    def parse_time(self,time,check_only=False,split=True,mseconds=True):
        ''' Return a parsing of the time
        '''
        try :
            time.hours
            return (time.days,time.hours,time.minutes,time.seconds,time.mseconds)
        except:
            pass
        if split:
            time=time.split(' ')[0]
        if time.count(',')==0 and not check_only and mseconds:
            raise()
        time=re.split("[:.,]+",time)
        if not mseconds:
            time=time+[0]
        while len(time)<4:
            time=[0]+time
        hours,minutes,seconds,mseconds=[float(x) for x in time]
        hours+=int(minutes/60)
        minutes=minutes%60
        days=int(hours/24)
        hours=hours%24
        if not check_only:
            return (days,hours,minutes,seconds,mseconds)
        
    def get_seconds(self):  
        return self.days*3600*24+self.hours*3600 + self.minutes*60 + self.seconds + self.mseconds/1000
    
    def __str__(self):
        return str(int(self.days))+';'+str(int(self.hours))+':'+str(int(self.minutes))+'.'+str(int(self.seconds))+','+str(int(self.mseconds))
    
    def __div__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()-seconds_other
    
    def __truediv__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()/seconds_other
    

In [ ]:
dict_problems={}

In [ ]:
def apply_time(x,mseconds=True):
    try:
        time=Time(x,mseconds=mseconds)
    except:
        try:
            dict_problems[x]+=1
        except:
            dict_problems[x]=1
        return False
    return time

In [ ]:
df.temps=df.temps.apply(apply_time)

In [ ]:
df.retard=df.retard.apply(apply_time)

In [ ]:
df.pace=df.pace.apply(apply_time,args=(False,))

In [ ]:
print(Time('95.12,2'))

In [ ]:
a=Time('48.12,2')
b=Time('4.30,0')
print(a,b)

In [ ]:
b.get_seconds()

In [ ]:
a.get_seconds()

In [ ]:
a/b

In [ ]:
df['Distance']=df.temps/df.pace

In [ ]:
df.head(2)

In [ ]:
df_old=df.copy()

In [ ]:
df=df[df.Distance>0]

In [ ]:
df_old.shape

In [ ]:
df=df_old[df_old.Distance>0]

In [ ]:
df.head(3)

# Parse Date

In [ ]:
df['Year']=df.Date.apply(lambda x: x.split('.')[3])

In [ ]:
df['Month']=df.Date.apply(lambda x: x.split('.')[2])

In [ ]:
df.head(2)

In [ ]:
df[(df.Name=='Jungfrau-Marathon, Interlaken') & (df.an=='1958') & (df.Year=='2012')]

# Sex Male-Female

### Get names database

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
gender_names=pd.read_csv('gender_names.csv')

In [ ]:
del gender_names['Unnamed: 0']

In [ ]:
gender_names=gender_names.set_index(gender_names.name)

In [ ]:
del gender_names['name']

In [ ]:
gender_names.head(2)

### Category from sex database

In [ ]:
category_list=list(df.cat.value_counts().index)

In [ ]:
len(category_list)

In [ ]:
category_dict_sex={}

In [ ]:
cat_to_check=['B','D','F40','F/3','H','12','10-F20']

In [ ]:
for i,cat in enumerate(category_list):
    if cat in category_dict_sex:
        continue
    names=list(df[df.cat==cat].nom.values)
    not_found=0
    male=0
    female=0
    for index,name in enumerate(names):
        splitted=re.split("[- ]+",name)
        found=0
        for name in splitted:
            try:
                row=gender_names.loc[name.lower()]
                male+=row.male_prob
                female+=row.female_prob
                found+=1
            except:
                pass
    if male!=0 or female!=0:
        male_perc=male/(male+female)
        category_dict_sex[cat]=male_perc
        if male_perc<0.85 and male_perc>0.15:
            if cat not in cat_to_check:
                cat_to_check.append(cat)
#             print('Category',cat,'Perc:',male_perc)
    if i%100==0:
#         print()
        print(i)
#         print()
    if i>7000:
        break

In [ ]:
len(cat_to_check)

In [ ]:
len(category_dict_sex)

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if category_dict_sex[cat]>0.85:
        cat_parsed+=1
        category_dict_sex[cat]='M'
    elif category_dict_sex[cat]<0.15:
        cat_parsed+=1
        category_dict_sex[cat]='F'

In [ ]:
category_dict_sex_copy=copy.deepcopy(category_dict_sex)

In [ ]:
import json

with open('category_dict_sex.json', 'w') as f:
    json.dump(category_dict_sex, f)

In [ ]:
import json

with open('category_dict_sex.json') as f:
    category_dict_sex = json.load(f)

### Improvements

In [ ]:
category_dict_sex=copy.deepcopy(category_dict_sex_copy)

In [ ]:
to_exclude=['P-W','D','SM2/D','W','P/W']

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if cat in to_exclude:
        continue
    if category_dict_sex[cat]=='M' or category_dict_sex[cat]=='F':
        cat_parsed+=1
        continue
    cat_splitted=re.split("[-/]+",cat)
    for i in cat_splitted:
        if len(i)==3 and (i[0]=='M' or i[0]=='H') and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        elif len(i)==3 and (i[0]=='F' or i[0]=='W')  and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Fille' in i or 'Women' in i or 'Dam' in i:
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Gar' in i or 'Men' in i or 'Hom' in i: 
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        if i=='W' or i=='D' or i=='H-U20W' or i=='H-WJ':
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
cat_parsed

In [ ]:
df[df.cat=='Sch-D-M']

### Final classification - accurate

In [ ]:
del df['sex']

In [ ]:
def add_sex(x):
    cat=x.cat.iloc[0]
    try:
        sex=category_dict_sex[cat]
        if sex=='M' or sex=='F':
            x['sex']=sex
        else:
            x['sex']=False
    except:
        x['sex']=False
    return x

In [ ]:
grouped=df.groupby(df.cat)

In [ ]:
df=grouped.apply(add_sex)

In [ ]:
df.sex.value_counts()

In [ ]:
df_sex=df[df.sex!=False]

In [ ]:
df=df_sex.copy()

In [ ]:
df_sex.head(2)

In [ ]:
df.shape

In [ ]:
df_sex.shape

In [ ]:
min(df.Year)

In [ ]:
min(df_sex.Year)

In [ ]:
df=df.rename(columns={'Year':'RaceYear','Month':'RaceMonth'})

In [ ]:
df.head(2)

In [ ]:
df.sex.value_counts()

# Parse year (roughly eliminating clearly bad entries)

In [ ]:
df.an=df.an.apply(lambda x: False if (x=='??' or x=='????' or x=='xxxx')  else x)

In [ ]:
df.an=df.an.apply(lambda x: int(x))

In [ ]:
df=df[df.an<2017]

In [ ]:
df=df[(df.an>1870) | (df.an<100)]

In [ ]:
df[(df.Name=='Jungfrau-Marathon, Interlaken') & (df.an==1958) & (df.RaceYear=='2012')]

# Columns names

In [ ]:
df.head(1)

In [ ]:
df.columns.values

In [ ]:
df=df[['Name','Date','RaceYear','RaceMonth','cat','Distance','nom','sex','an','lieu','rang','temps','retard','pace']]

In [ ]:
df.head()

In [ ]:
df.columns=['Race','Date','RaceYear','RaceMonth','Category','Distance','Name','Sex','Year','LivingPlace','Rank','Time','Delay','Pace']

In [ ]:
df=df.reset_index()

In [ ]:
del df['index']

In [ ]:
df.head(2)

In [ ]:
df.shape

# Distance

In [ ]:
df.Distance=df.Distance.apply(lambda x: float(x))

In [ ]:
df['round']=df.Distance.round()

In [ ]:
df.Distance=df.Distance.apply(lambda x: x if (abs(x-round(x)))>0.065 else round(x))

In [ ]:
df.Distance=df.Distance.apply(lambda x: 21.0975 if (abs(x-21.0975))<0.1 else x)

In [ ]:
df.Distance=df.Distance.apply(lambda x: 42.195 if (abs(x-42.195))<0.1 else x)

In [ ]:
df.Distance.value_counts()

In [ ]:
del df['round']

In [ ]:
df.head(5)

In [ ]:
df.shape

# Adding weather informations

In [ ]:
weather_db=pd.read_csv('../datasets/races-information-weather.csv')
del weather_db['Unnamed: 0']
del weather_db['Unnamed: 0.1']

In [ ]:
weather_db.head(2)

In [ ]:
weather_db['RaceYear']=weather_db.Date.apply(lambda x: x.split('.')[3])

In [ ]:
gr=df.groupby([df.RaceYear,df.Race])

In [ ]:
def merge_weather_info(x):
    race=x.iloc[0].Race
    date=x.iloc[0].RaceYear
    weather=weather_db[(weather_db.Name==race) & (weather_db.RaceYear==date)].iloc[0]
    x['Place']=weather.Place
    x['MinTemp']=weather.min_temp
    x['MaxTemp']=weather.max_temp
    x['Weather']=weather.weather_desc
    x['RaceID']=weather.URL
    
    return x

In [ ]:
df=gr.apply(merge_weather_info)

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

# Parse times and select only meaningful features

In [ ]:
full_df=df.copy()

In [ ]:
base=datetime.datetime.strptime('0:0.0,0','%H:%M.%S,%f')

In [ ]:
full_df.Time=full_df.Time.apply(lambda x:str(x))

In [ ]:
full_df.Pace=full_df.Pace.apply(lambda x:str(x))

In [ ]:
def modify_to_parse_day(x):
    a=str(int(x[0])+1)
    return a+x[1:]

In [ ]:
full_df.Pace=full_df.Pace.apply(modify_to_parse_day)

In [ ]:
full_df.Time=full_df.Time.apply(modify_to_parse_day)

In [ ]:
full_df['Pace'] = (pd.to_datetime(full_df.Pace, format='%d;%H:%M.%S,%f')-base)

In [ ]:
max(full_df.Time)

In [ ]:
max(full_df.Distance)

In [ ]:
full_df['Time'] = (pd.to_datetime(full_df.Time, format='%d;%H:%M.%S,%f')-base)

In [ ]:
del full_df['Delay']

In [ ]:
full_df.head()

# Better distance processing

In [ ]:
df.Race.value_counts()

In [ ]:
full_df2=full_df.copy()

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: x if x<4 or (abs(1-x/round(x)))>0.0055 else round(x))

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 21.0975 if x>4 and (abs(1-x/21.0975))<0.0055 else x)

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 42.195 if x>4 and (abs(1-x/42.195))<0.0055 else x)

In [ ]:
full_df2.groupby(full_df2.RaceYear).count()

In [ ]:
df=full_df2.copy()

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

# Year - trying to get better data

In [ ]:
years_values=df.Year.value_counts().index.values
years_values[years_values<120]

In [ ]:
df.Year=df.Year.apply(lambda x: 0 if x==2 else x)

In [ ]:
df[df.Year>1900].shape

In [ ]:
df[df.Year<1900].shape

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
reduced_df[reduced_df.Year==25]

In [ ]:
reduced_df.groupby(reduced_df.Year).count()

If in the name a comma is present, it is possible that the year is misunderstood.

In [ ]:
names=reduced_df.Name
list_names=[]
for name in names:
    if ',' in name:
        list_names.append(name)

In [ ]:
df['CommaInName']=df.Name.str.contains(',')

In [ ]:
df['Less120']=df.Year<120

In [ ]:
df['FalseYear']=df.CommaInName & df.Less120

In [ ]:
def false_year(x):
    if x.iloc[0].FalseYear:
        x['Year']=float('nan')
    return x

In [ ]:
df=df.groupby(df.FalseYear).apply(false_year)

In [ ]:
del df['CommaInName']
del df['Less120']
del df['FalseYear']

In [ ]:
df.head(4)

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
list(reduced_df.Year.value_counts().index)

In [ ]:
df.Year=df.Year.apply(lambda x: float('nan') if x<10 else x)

In [ ]:
min(df.Year)

In [ ]:
df.Year=df.Year.apply(lambda x: x+1900 if x<120 else x)

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

# Output
First output

In [ ]:
df_output=df.copy()

In [ ]:
df_output.shape

In [ ]:
df_output.head()

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split(',')[0])

In [ ]:
df_output['len_name']=df_output.Name.apply(lambda x: len(x.rstrip().lstrip().split(' ')))

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split('+')[0])

In [ ]:
df_output

In [ ]:
df_output.len_name.value_counts()

In [ ]:
df_output[df_output.len_name==6]

In [ ]:
df_output.groupby(df_output.RaceYear).count()

In [ ]:
df.groupby(df.RaceYear).count()

In [ ]:
df_output.to_pickle('../datasets/intermediate_dataset.pickle')

# Solve problem 2 runners with same name in the same race

Strategy - study duplicates:  
- Remove all fully duplicated entries (same player in different categories)
- Keep entries with different living place (different players)
- To decide what to do if same place and different times

In [ ]:
df_output.Race.value_counts()

In [5]:
df_output=pd.read_pickle('../datasets/intermediate_dataset.pickle')

In [6]:
df=pd.read_pickle('../datasets/intermediate_dataset.pickle')

In [ ]:
df_output[(df_output.Race=='Jungfrau-Marathon, Interlaken') & (df_output.Year==1958) & (df_output.LivingPlace=='Ostermundigen')]

In [ ]:
df_red=df_output.copy()#[df_output.Race=='Course de l\'Escalade, Genève']

In [ ]:
df_red.shape

In [ ]:
grouped=df_red.groupby([df_red.RaceID,df_red.Name,df_output.Year])

In [ ]:
duplicates=grouped.filter(lambda x: len(x)>1)

In [ ]:
duplicates.shape

In [ ]:
duplicates

In [ ]:
duplicates.Race.value_counts()

In [ ]:
a=df_output[(df_output.Race=='Jungfrau-Marathon, Interlaken') & (df_output.RaceYear==2012)]

In [ ]:
a[a.Name.to_series().str.contains('Baumer').tolist()]

In [ ]:
a[a.Name=='Peter Baumer']

In [ ]:
df_output[df_output.Name=='Baumer Peter']

In [ ]:
df_output.Race.value_counts()

In [ ]:
df_output[(df_output.Name=='Baumer Peter')]

In [ ]:
duplicates.columns

In [ ]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [ ]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)]

In [ ]:
d2.shape

In [ ]:
d2[d2.Race=='Zürcher Silvesterlauf, Zürich']

In [ ]:
d2.Distance.value_counts()

In [ ]:
d1=duplicates[duplicates.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [ ]:
d1.shape

In [ ]:
d1.Race.value_counts()

In [ ]:
4961-4580

In [ ]:
duplicates.shape

In [ ]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

In [ ]:
dup.shape

In [ ]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth','Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)

In [ ]:
dup.shape

In [ ]:
dup[dup.Race=='Jungfrau-Marathon, Interlaken']

In [8]:
df_output=df_output.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

In [9]:
df_output[(df_output.Race=='Jungfrau-Marathon, Interlaken') & (df_output.RaceYear=='2012') & (df_output.Year==1958)]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name
1105423,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Affolter Rosmarie,F,1958.0,Muntelier,197,06:30:41.800000,00:09:15,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105431,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-M50,42.195,Ailinger Thomas,M,1958.0,Pratteln,470,05:29:57.500000,00:07:49,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105529,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Appleby Judy,F,1958.0,GB-Redditch Worcestershir,68,05:23:58.600000,00:07:40,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105539,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-M50,42.195,Arnold Johannes,M,1958.0,DE-Hockenheim,826,07:00:02.800000,00:09:57,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105598,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Bajerova Ilona,F,1958.0,CZ-Tupesy,53,05:18:22.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105658,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Baschung Regina,F,1958.0,Hinterkappelen,34,05:04:48.400000,00:07:13,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105695,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,117,05:14:16.400000,00:07:26,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105697,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,128,05:18:23.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105719,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-M50,42.195,Bäuerle Werner,M,1958.0,DE-Konstanz,516,05:36:23.900000,00:07:58,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105737,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-M50,42.195,Becker Axel,M,1958.0,DE-Augsburg,129,04:40:17.600000,00:06:38,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2


In [7]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name
1105423,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Affolter Rosmarie,F,1958.0,Muntelier,197,06:30:41.800000,00:09:15,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105431,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-M50,42.195,Ailinger Thomas,M,1958.0,Pratteln,470,05:29:57.500000,00:07:49,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105529,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Appleby Judy,F,1958.0,GB-Redditch Worcestershir,68,05:23:58.600000,00:07:40,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105539,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-M50,42.195,Arnold Johannes,M,1958.0,DE-Hockenheim,826,07:00:02.800000,00:09:57,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105598,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Bajerova Ilona,F,1958.0,CZ-Tupesy,53,05:18:22.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105658,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-F50,42.195,Baschung Regina,F,1958.0,Hinterkappelen,34,05:04:48.400000,00:07:13,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105695,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,117,05:14:16.400000,00:07:26,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105696,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SA-M50,42.195,Baumer Peter,M,1958.0,Ostermundigen,361,05:14:16.400000,00:07:26,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105697,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,128,05:18:23.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105698,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,09,SO-M50,42.195,Baumer Peter,M,1958.0,Ostermundigen,25,05:18:23.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2


# Sex verification
The sex classification seems working

In [ ]:
lau=df_output[(df_output.RaceID=='http://services.datasport.com/2007/lauf/Greifenseelauf')&(df_output.Distance>6)]

In [ ]:
lau.Distance.value_counts()

In [ ]:
lau.Sex.value_counts()

In [ ]:
df_output.RaceID.value_counts()

In [ ]:
df_output.shape

# Fixing not good categories

In [10]:
df_output.shape

(1650887, 19)

In [37]:
df=df_output.copy()

In [38]:
df.shape

(1650887, 18)

In [39]:
cat_list=list(df.Category.value_counts().index)

In [42]:
df[df.Category=='MTBM1'].head(2)

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
7499,"Gurtenclassic, Wabern",dim. 05.11.2000,2000,11,MTBM1,10.0,Adam Benjamin,M,1970.0,Thun,46,00:37:57.700000,00:03:47,Wabern,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/gurten
7503,"Gurtenclassic, Wabern",dim. 05.11.2000,2000,11,MTBM1,10.0,Aebi Sven,M,1976.0,Bern,66,00:42:16.900000,00:04:13,Wabern,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/gurten


In [43]:
cat_to_eliminate=[]
for i in cat_list:
    if 'MTB' in i:
        print(i)
        cat_to_eliminate.append(i)

MTBM1
MTBM2
MTBM3
MTBMJ
MTBKSM
MTBW
MTBMSM


In [44]:
for i in cat_to_eliminate:
    df=df[df.Category!=i]

In [45]:
df.shape

(1650657, 18)

In [46]:
df_output=df.copy()

In [48]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear==2012) & (df.Year==1958)]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
1105331,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Affolter Rosmarie,F,1958.0,Muntelier,197,06:30:41.800000,00:09:15,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105339,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Ailinger Thomas,M,1958.0,Pratteln,470,05:29:57.500000,00:07:49,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105437,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Appleby Judy,F,1958.0,GB-Redditch Worcestershir,68,05:23:58.600000,00:07:40,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105447,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Arnold Johannes,M,1958.0,DE-Hockenheim,826,07:00:02.800000,00:09:57,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105506,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Bajerova Ilona,F,1958.0,CZ-Tupesy,53,05:18:22.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105558,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Baschung Regina,F,1958.0,Hinterkappelen,34,05:04:48.400000,00:07:13,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105595,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,117,05:14:16.400000,00:07:26,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105596,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,128,05:18:23.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105617,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Bäuerle Werner,M,1958.0,DE-Konstanz,516,05:36:23.900000,00:07:58,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau
1105635,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Becker Axel,M,1958.0,DE-Augsburg,129,04:40:17.600000,00:06:38,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau


# Capitalization

In [22]:
'CIAO COME STAI'.title()

'Ciao Come Stai'

In [23]:
df=df_output.copy()

In [24]:
df.dtypes

Race                    object
Date                    object
RaceYear                object
RaceMonth               object
Category                object
Distance               float64
Name                    object
Sex                     object
Year                   float64
LivingPlace             object
Rank                     int64
Time           timedelta64[ns]
Pace           timedelta64[ns]
Place                   object
MinTemp                float64
MaxTemp                float64
Weather                 object
RaceID                  object
len_name                 int64
dtype: object

In [25]:
df.RaceYear=df.RaceYear.apply(lambda x: int(x))

In [26]:
df.RaceMonth=df.RaceMonth.apply(lambda x: int(x))

In [27]:
df.Name=df.Name.apply(lambda x: x.title())

In [28]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

In [29]:
df=df.reset_index()
del df['index']

In [31]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear==2012) & (df.Year==1958)]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name
1105331,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Affolter Rosmarie,F,1958.0,Muntelier,197,06:30:41.800000,00:09:15,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105339,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Ailinger Thomas,M,1958.0,Pratteln,470,05:29:57.500000,00:07:49,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105437,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Appleby Judy,F,1958.0,GB-Redditch Worcestershir,68,05:23:58.600000,00:07:40,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105447,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Arnold Johannes,M,1958.0,DE-Hockenheim,826,07:00:02.800000,00:09:57,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105506,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Bajerova Ilona,F,1958.0,CZ-Tupesy,53,05:18:22.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105558,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-F50,42.195,Baschung Regina,F,1958.0,Hinterkappelen,34,05:04:48.400000,00:07:13,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105595,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,117,05:14:16.400000,00:07:26,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105596,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,DoppelM,42.195,Baumer Peter,M,1958.0,Ostermundigen,128,05:18:23.800000,00:07:32,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105617,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Bäuerle Werner,M,1958.0,DE-Konstanz,516,05:36:23.900000,00:07:58,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2
1105635,"Jungfrau-Marathon, Interlaken",sam. 08.09.2012,2012,9,SA-M50,42.195,Becker Axel,M,1958.0,DE-Augsburg,129,04:40:17.600000,00:06:38,Interlaken,8.0,21.0,Clear,http://services.datasport.com/2012/lauf/jungfrau,2


In [49]:
df_output=df.copy()

# Fix names

In [69]:
df_output.Name=df_output.Name.apply(lambda x: x.rstrip())

In [70]:
df_output.Name=df_output.Name.apply(lambda x: x.lstrip())

In [71]:
df_output[df_output.Name=='Baumer Peter']

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
201,Kerzerslauf,sam. 18.03.2000,2000,3,M40,15.000000,Baumer Peter,M,1958.0,Münsingen,265,01:12:30.300000,00:04:50,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
26991,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,9,M40,42.195000,Baumer Peter,M,1958.0,Münsingen,372,05:03:14.500000,00:07:11,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau
126521,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,2003,9,M45,42.195000,Baumer Peter,M,1958.0,Münsingen,292,05:11:02.600000,00:07:22,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/jungfrau
157064,Kerzerslauf,sam. 20.03.2004,2004,3,M45,15.000000,Baumer Peter,M,1958.0,Münsingen,277,01:16:11,00:05:04,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2004/lauf/kerzers
199145,"Jungfrau-Marathon, Interlaken",sam. 11.09.2004,2004,9,M45,42.195000,Baumer Peter,M,1958.0,Münsingen,417,05:39:41,00:08:03,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2004/lauf/jungfrau
217150,Kerzerslauf,sam. 19.03.2005,2005,3,M45,15.000000,Baumer Peter,M,1958.0,Münsingen,221,01:14:43.400000,00:04:58,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/kerzers
224461,"Chäsitzerlouf, Kehrsatz",sam. 23.04.2005,2005,4,M40,12.340440,Baumer Peter,M,1958.0,Bern,147,00:58:00.400000,00:04:42,Kehrsatz,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/kehrsatz
262831,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,2005,9,M45,42.195000,Baumer Peter,M,1958.0,Bern,168,04:43:23.100000,00:06:42,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/jungfrau
276508,"Gurtenclassic, Wabern",dim. 06.11.2005,2005,11,M40,15.000000,Baumer Peter,M,1958.0,Bern,135,01:22:46.300000,00:05:31,Wabern,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/gurten
280087,"Course de l'Escalade, Genève",sam. 03.12.2005,2005,12,Hommes3,7.260000,Baumer Peter,M,1958.0,BERN,1170,00:36:18,00:05:00,Genève,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/escalade


# Try to identify different people with same name

- Remove race Töss-Stafette, Winterthur
- Drop the duplicates with same userID, race and distance (same player, tipycally error in Datasport). In case of Jungfrau marathon I keep only 1 result.
- Keep the longest race if same userID and race but different distance

In [271]:
df_output=pd.read_pickle('../datasets/df_added_category.pickle')

In [272]:
df_output.LivingPlace=df_output.LivingPlace.apply(lambda x: x.lstrip())
df_output.LivingPlace=df_output.LivingPlace.apply(lambda x: x.rstrip())

In [273]:
df_output=df_output[df_output.Race!='Töss-Stafette, Winterthur']

In [274]:
df=df_output.copy()

In [275]:
df['YearStr']=df.Year.apply(lambda x: str(x))

In [276]:
df['UserID']=df.Name+' '+df.YearStr+' '+df.LivingPlace

In [277]:
df['UserIDYear']=df.Name+' '+df.YearStr

In [278]:
df['DistanceRound']=df.Distance.apply(lambda x: round(x))

In [279]:
df=df.drop_duplicates(subset=['RaceID','UserID','DistanceRound'])

In [280]:
df[df.Year.isnull()].shape

(36663, 22)

In [304]:
df[df.duplicated(subset=['RaceID','Name','Year'])].shape

(652, 22)

In [305]:
for i in list(df.LivingPlace.value_counts().index):
    if 'Reinach' in i:
        print(i)

Reinach BL
Reinach AG
Reinach
Reinach BASEL
CH-Reinach BL
Sek Reinach
Reinach BL 2
VC Reinach Goldwurstpower
Sportklasse/KV Reinach
Ski Club Reinach
VC Reinach
VC Reinach/goldwurst-powe
Sekundarschule Reinach
Reinach 1
SC Reinach
Menzo Reinach
FC Reinach


In [298]:
list(df.LivingPlace.value_counts().index)

['Zürich',
 'Lausanne',
 'Bern',
 'Genève',
 'Basel',
 'Winterthur',
 'Luzern',
 'Fribourg',
 'GENEVE',
 'Uster',
 'Neuchâtel',
 'Thun',
 'La Chaux-de-Fonds',
 'Kerzers',
 'St. Gallen',
 'Pully',
 'Bulle',
 'Riehen',
 'Biel/Bienne',
 'Burgdorf',
 'Carouge GE',
 'Steffisburg',
 'Thalwil',
 'Dübendorf',
 'Muttenz',
 'Zug',
 'Wallisellen',
 'Allschwil',
 'Kriens',
 'Reinach BL',
 'Epalinges',
 'Adliswil',
 'Villars-sur-Glâne',
 'Nyon',
 'Yverdon-les-Bains',
 'Morges',
 'Aarau',
 'Binningen',
 'Wettingen',
 'Chur',
 'Baar',
 'Murten',
 'Frauenfeld',
 'Dietikon',
 'Horgen',
 'Liestal',
 'Ostermundigen',
 'Kloten',
 'Monthey',
 'Renens VD',
 'Therwil',
 'Baden',
 'Bülach',
 'Oberwil BL',
 'Düdingen',
 'Rothenburg',
 'Emmenbrücke',
 'Colombier NE',
 'Wetzikon ZH',
 'Gland',
 'Ecublens VD',
 'Jona',
 'Volketswil',
 'Solothurn',
 'Wädenswil',
 'Langenthal',
 'Petit-Lancy',
 'Münsingen',
 'Sion',
 'Vevey',
 'Prilly',
 'Grand-Lancy',
 'Belp',
 'Cham',
 'Thônex',
 'Bassersdorf',
 'La Tour-de-Peilz

In [281]:
len(set(df.Name))

466389

In [282]:
len(set(df.UserIDYear))

531426

In [283]:
len(set(df.UserID))

686131

In [284]:
df=df[df.groupby([df.RaceID,df.UserID])['Distance'].transform(max)==df['Distance']]

### Process LivingPlace

In [729]:
cantons=['AG','AI','AR','BE','BL','BS','FR','GE','GL','GR','JU','LU','NE','NW','OW','SG',
         'SH','SO','SZ','TG','TI','UR','VD','VS','ZG','ZH','GB','TV','AT','KLH','DK','DE',
         'JP','NL','VA','BE','RC','MI','BG','UBS','NO','RN','BZ','LC','FI','CO','UD','TO','MB',
        'RE','VB','US','SV','AL','PO','BO','NA','BI','SE','PT','LO','AO','FL','DC','LR','LA','SR',
         'ZS']

In [730]:
living_places=list(df.LivingPlace.value_counts().index)
proc1=living_places[:]

In [731]:
for i,word in enumerate(living_places):
    if word.isupper():
        proc1[i]=word.title()

In [732]:
processing_dict={i:j for i,j in zip(living_places,proc1)}

In [733]:
for cat1,cat2 in zip(living_places,proc1):
    for st in cantons:
        if st in cat2:
            processing_dict[cat1]=cat2.replace(st,'').lstrip().rstrip()
    processing_dict[cat1]=processing_dict[cat1].replace('-',' ').lstrip().rstrip()
    processing_dict[cat1]=processing_dict[cat1].replace('(','')
    processing_dict[cat1]=processing_dict[cat1].replace(')','')

In [734]:
b=df.copy()

In [735]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [736]:
processing_dict={i:i for i in processing_dict.values()}

In [737]:
living_places=list(b.LivingPlace.value_counts().index)

In [738]:
living_places_start=living_places[:100]
living_places=living_places[100:]

In [739]:
for i in living_places_start:
    for j in living_places:
        if i in j:
            processing_dict[j]=i

In [740]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [741]:
processing_dict={i:i for i in processing_dict.values()}

In [742]:
for i in processing_dict.keys():
    try:
        if i[1]==' ':
            processing_dict[i]=processing_dict[i][2:].lstrip()
    except:
        pass

In [743]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [744]:
processing_dict={i:i for i in processing_dict.values()}

In [745]:
len(list(b.LivingPlace.value_counts().index))

46857

In [746]:
living_places=list(b.LivingPlace.value_counts().index)

In [747]:
living_places_start=living_places[:5000]
living_places=living_places[5000:]

In [748]:
for i in living_places_start:
    for j in living_places:
        if i in j and len(i)>1:
            processing_dict[j]=i

In [749]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [750]:
processing_dict={i:i for i in processing_dict.values()}

In [751]:
len(list(b.LivingPlace.value_counts().index))

35827

In [752]:
import unicodedata
for i in processing_dict.keys():
    s=processing_dict[i]
    processing_dict[i]=''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [753]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [754]:
processing_dict={i:i for i in processing_dict.values()}

In [755]:
len(list(b.LivingPlace.value_counts().index))

35550

In [756]:
values=list(b.LivingPlace.value_counts().values)

In [757]:
living_places=list(b.LivingPlace.value_counts().index)

In [763]:
b.shape

(1648694, 22)

In [765]:
sum(values[:5000])

1563648

In [764]:
glob=[(i,j) for i,j in zip(living_places,values)]
glob[9000:10000]

[('Hillerod', 6),
 ('Sassenage', 6),
 ('Footing Vallee de Joux 10', 6),
 ("'s Hertogenbosch", 6),
 ('running saronno', 6),
 ('Yellow Kittys', 6),
 ('Bubbiano ', 6),
 ('Angelholm', 6),
 ('Union Cycliste Internatio', 6),
 ('Asics', 6),
 ('Hochemmingen', 6),
 ('Fußach', 6),
 ('Palma de Mallorca', 6),
 ('Napoli ', 6),
 ("Ooo's", 6),
 ('Nord Stream', 6),
 ('Dongo', 6),
 ('Lions Running Team 08', 6),
 ('Calida', 6),
 ('Team Nanga Parbat', 6),
 ('Rebmann Elektro', 6),
 ('Koblach', 6),
 ('Skylab Team', 6),
 ('SC Ebmattingen', 6),
 ('ol norska', 6),
 ('Meudon', 6),
 ('Ercos Management SA', 6),
 ('Le Vully', 6),
 ('Running Team', 6),
 ('fun and ran', 6),
 ('Havixbeck', 6),
 ('Hoeri', 6),
 ('Wertheim', 6),
 ('Team Saucony', 6),
 ('Naumburg', 6),
 ('Fussen', 6),
 ('Jpf Constructions Sa', 6),
 ('Lauchhammer', 6),
 ('TL 20 km 3', 6),
 ('Sellenburen', 6),
 ('Equipe Liaudet Pial SA', 6),
 ('Ezone challenge4charity', 6),
 ('St Albans', 6),
 ('Aists 2', 6),
 ('Rudersberg', 6),
 ('Taino/Varese', 6),
 ('H

In [721]:
for i in living_places:
    if 'Neuchatel' in i:
        print(i)

Neuchatel


In [285]:
b=df.copy()

In [296]:
b[b.Name=='Belluschi Elio']

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,...,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,YearStr,UserID,UserIDYear,DistanceRound
4147,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:00,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
14678,"Giro Media Blenio, Dongio",lun. 16.04.2001,2001,4,11,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:02,Dongio,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
97725,"Giro Media Blenio, Dongio",lun. 21.04.2003,2003,4,11,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:09,Dongio,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
167883,"Giro Media Blenio, Dongio",lun. 12.04.2004,2004,4,11,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:09,Dongio,NaN,NaN,NaN,http://services.datasport.com/2004/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
216099,"Giro Media Blenio, Dongio",lun. 28.03.2005,2005,3,11,10.0,Belluschi Elio,M,NaN,-Italia,...,00:03:20,Dongio,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/blenio,nan,Belluschi Elio nan -Italia,Belluschi Elio nan,10
309670,"Giro Media Blenio, Dongio",lun. 17.04.2006,2006,4,11,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:19,Dongio,NaN,NaN,NaN,http://services.datasport.com/2006/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
405243,"Giro Media Blenio, Dongio",lun. 09.04.2007,2007,4,11,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:16,Dongio,NaN,NaN,NaN,http://services.datasport.com/2007/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
490269,"Giro Media Blenio, Dongio",lun. 24.03.2008,2008,3,11,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:21,Dongio,NaN,NaN,NaN,http://services.datasport.com/2008/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
622334,"Giro Media Blenio, Dongio",lun. 13.04.2009,2009,4,GP,10.0,Belluschi Elio,M,NaN,Italia,...,00:03:24,Dongio,1.0,3.0,Fog,http://services.datasport.com/2009/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10


In [288]:
year_null=b[b.Year.isnull()]

In [292]:
year_null[year_null.duplicated(subset=['UserID'],keep=False)]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,...,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,YearStr,UserID,UserIDYear,DistanceRound
4147,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Belluschi Elio,M,NaN,Italia,...,00:03:00,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Belluschi Elio nan Italia,Belluschi Elio nan,10
4148,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Bernardini Severino,M,NaN,Italia,...,00:03:05,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Bernardini Severino nan Italia,Bernardini Severino nan,10
4152,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Chiesa Sergio,M,NaN,Italia,...,00:03:04,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Chiesa Sergio nan Italia,Chiesa Sergio nan,10
4153,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Cogno Fabrizio,M,NaN,Italia,...,00:03:12,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Cogno Fabrizio nan Italia,Cogno Fabrizio nan,10
4156,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Foglia Luca,M,NaN,Svizzera,...,00:03:16,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Foglia Luca nan Svizzera,Foglia Luca nan,10
4157,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Ivuti Patrick,M,NaN,Kenya,...,00:02:54,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Ivuti Patrick nan Kenya,Ivuti Patrick nan,10
4159,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Kipchumba Robert,M,NaN,Kenya,...,00:02:54,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Kipchumba Robert nan Kenya,Kipchumba Robert nan,10
4161,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Kosgei Paul,M,NaN,Kenya,...,00:02:54,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Kosgei Paul nan Kenya,Kosgei Paul nan,10
4166,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Pongelli Ivan,M,NaN,Svizzera,...,00:03:08,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Pongelli Ivan nan Svizzera,Pongelli Ivan nan,10
4168,"Giro Media Blenio, Dongio",lun. 24.04.2000,2000,4,11,10.000000,Stampanoni Jonathan,M,NaN,Svizzera,...,00:03:11,Dongio,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/blenio,nan,Stampanoni Jonathan nan Svizzera,Stampanoni Jonathan nan,10


In [289]:
year_null.RaceYear.value_counts()

2006    7703
2003    6355
2005    5822
2002    5650
2004    5354
2001    2986
2007    1260
2000     449
2008     307
2009     243
2010     130
2011     110
2012      98
2014      69
2013      64
2015      28
Name: RaceYear, dtype: int64

In [186]:
del gr_duplicated['YearStr']

In [268]:
gr_duplicated=b[b.duplicated(subset=['RaceID','UserID'],keep=False)]

In [269]:
gr_duplicated.shape

(0, 22)

In [230]:
gr_duplicated[gr_duplicated.Race=='Hellebardenlauf Sempach']

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,...,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,YearStr,UserID,DistanceRound,UserIDYear
108495,Hellebardenlauf Sempach,sam. 28.06.2003,2003,6,DK,1.000000,Renggli Sandro,M,1993.0,Ballwil,...,00:04:09,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/sempach,1993.0,Renggli Sandro 1993.0 Ballwil,1,Renggli Sandro 1993.0
108496,Hellebardenlauf Sempach,sam. 28.06.2003,2003,6,5/A,5.611486,Renggli Sandro,M,1993.0,Ballwil,...,00:04:56,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/sempach,1993.0,Renggli Sandro 1993.0 Ballwil,6,Renggli Sandro 1993.0
108761,Hellebardenlauf Sempach,sam. 28.06.2003,2003,6,5/A,5.615141,Wey Yannick,M,NaN,Schule Tann,...,00:04:51,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/sempach,nan,Wey Yannick nan Schule Tann,6,Wey Yannick nan
108762,Hellebardenlauf Sempach,sam. 28.06.2003,2003,6,DK,1.000000,Wey Yannick,M,NaN,Schule Tann,...,00:03:49,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/sempach,nan,Wey Yannick nan Schule Tann,1,Wey Yannick nan
233870,Hellebardenlauf Sempach,sam. 25.06.2005,2005,6,1km/JKC,1.000000,Arnold Mark,M,NaN,Läuferriege Neuenkirch,...,00:03:46,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/sempach,nan,Arnold Mark nan Läuferriege Neuenkirch,1,Arnold Mark nan
233871,Hellebardenlauf Sempach,sam. 25.06.2005,2005,6,5.8k/JKA,5.817121,Arnold Mark,M,NaN,Läuferriege Neuenkirch,...,00:04:17,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/sempach,nan,Arnold Mark nan Läuferriege Neuenkirch,6,Arnold Mark nan
233972,Hellebardenlauf Sempach,sam. 25.06.2005,2005,6,1km/JKD,1.000000,Bucher Kim,M,NaN,Schule Tann,...,00:04:28,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/sempach,nan,Bucher Kim nan Schule Tann,1,Bucher Kim nan
233973,Hellebardenlauf Sempach,sam. 25.06.2005,2005,6,5.8k/JKA,5.803941,Bucher Kim,M,NaN,Schule Tann,...,00:05:57,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/sempach,nan,Bucher Kim nan Schule Tann,6,Bucher Kim nan
234004,Hellebardenlauf Sempach,sam. 25.06.2005,2005,6,1km/JKC,1.000000,Bürgisser Michael,M,NaN,STV Sempach,...,00:03:54,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/sempach,nan,Bürgisser Michael nan STV Sempach,1,Bürgisser Michael nan
234005,Hellebardenlauf Sempach,sam. 25.06.2005,2005,6,5.8k/JKA,5.809884,Bürgisser Michael,M,NaN,STV Sempach,...,00:04:44,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/sempach,nan,Bürgisser Michael nan STV Sempach,6,Bürgisser Michael nan


# Output to file

In [33]:
del df_output['len_name']

In [72]:
df_output.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID'],
      dtype='object')

In [73]:
df_output

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
0,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Abgottspon Peter,M,1974.0,Zermatt,233,01:02:25,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
1,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Abplanalp Michael,M,1964.0,Bern,32,00:55:11.700000,00:03:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
2,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Abt Werner,M,1947.0,Spiez,155,01:12:42.900000,00:04:50,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
3,Kerzerslauf,sam. 18.03.2000,2000,3,F45,15.000000,Ackermann Antoinette,F,1953.0,Alterswil,48,01:22:36.700000,00:05:30,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
4,Kerzerslauf,sam. 18.03.2000,2000,3,F50,15.000000,Ackermann Hedy,F,1946.0,Alterswil FR,42,01:23:29.300000,00:05:33,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
5,Kerzerslauf,sam. 18.03.2000,2000,3,M40,15.000000,Ackermann Meinrad,M,1956.0,La Joux,35,00:56:53.100000,00:03:47,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
6,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adamina Bruno,M,1962.0,Bern,26,00:54:41.700000,00:03:38,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
7,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adatte Michel,M,1964.0,Asuel,48,00:56:36.600000,00:03:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
8,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Adesris Yeshaneh,M,1973.0,ETH-Aethiopien,23,00:49:27.300000,00:03:17,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
9,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Adler Hanspeter,M,1949.0,Koblenz,33,01:02:23,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers


In [74]:
df_output.shape

(1650657, 18)

In [76]:
df_output.dtypes

Race                    object
Date                    object
RaceYear                 int64
RaceMonth                int64
Category                object
Distance               float64
Name                    object
Sex                     object
Year                   float64
LivingPlace             object
Rank                     int64
Time           timedelta64[ns]
Pace           timedelta64[ns]
Place                   object
MinTemp                float64
MaxTemp                float64
Weather                 object
RaceID                  object
dtype: object

In [77]:
df_output.shape

(1650657, 18)

In [82]:
df_output.to_pickle('../datasets/df_added_category.pickle')